In [17]:
import numpy as np
import pandas as pd

In [18]:
open = pd. read_csv('email_opened_table.csv')
click = pd. read_csv('link_clicked_table.csv')
email = pd. read_csv('email_table.csv')

In [19]:
open

,email_id
0,284534
1,609056
2,220820
3,905936
4,164034
...,...
10340,990213
10341,918210
10342,200299
10343,211104


In [20]:
click

,email_id
0,609056
1,870980
2,935124
3,158501
4,177561
...,...
2114,873162
2115,435454
2116,206772
2117,742967


In [21]:
email

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases
0,85120,short_email,personalized,2,Sunday,US,5
1,966622,long_email,personalized,12,Sunday,UK,2
2,777221,long_email,personalized,11,Wednesday,US,2
3,493711,short_email,generic,6,Monday,UK,1
4,106887,long_email,generic,14,Monday,US,6
...,...,...,...,...,...,...,...
99995,803504,short_email,personalized,4,Monday,US,5
99996,899722,long_email,personalized,7,Saturday,US,1
99997,449610,long_email,personalized,11,Saturday,UK,6
99998,72497,short_email,generic,10,Monday,UK,0


In [22]:
open['opened'] = 1
click["clicked"] = 1

In [23]:
df1= pd.merge(open, click, on = 'email_id', how = 'outer')
df= pd.merge(df1, email, on = 'email_id', how = 'outer')

In [24]:
df['opened'] = df['opened'].fillna(0).astype(int)
df['clicked'] = df['clicked'].fillna(0).astype(int)

In [25]:
((df['opened'] == 0) & (df['clicked'] == 1)).sum()

np.int64(50)

In [26]:
anomaly = (df['opened'] == 0) & (df['clicked'] == 1)
df = df[~anomaly]

In [27]:
def engagement(row):
    if (row['opened'] == 1) and (row['clicked'] == 1):
        return 'both'
    elif (row['opened'] == 1) and (row['clicked'] == 0):
        return 'opened'
    elif (row['opened'] == 0) and (row['clicked'] == 1):
        return 'clicked-->outlier' 
    elif (row['opened'] == 0) and (row['clicked'] == 0):
        return 'none'
    else:
        return 'unknown' 

In [28]:
df['target_col'] = df.apply(engagement, axis=1)

In [29]:
df

,email_id,opened,clicked,email_text,email_version,hour,weekday,user_country,user_past_purchases,target_col
0,8,0,0,short_email,generic,9,Thursday,US,3,none
1,33,0,0,long_email,personalized,6,Monday,US,0,none
2,46,0,0,short_email,generic,14,Tuesday,US,3,none
3,49,1,0,long_email,personalized,11,Thursday,US,10,opened
4,65,0,0,short_email,generic,8,Wednesday,UK,3,none
...,...,...,...,...,...,...,...,...,...,...
99995,999969,0,0,short_email,generic,21,Thursday,US,0,none
99996,999972,0,0,long_email,personalized,6,Tuesday,US,5,none
99997,999976,0,0,long_email,personalized,5,Wednesday,UK,0,none
99998,999980,0,0,long_email,generic,10,Thursday,FR,4,none


In [30]:
df.drop(columns=['opened', 'clicked'], inplace=True)

In [31]:
df

,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases,target_col
0,8,short_email,generic,9,Thursday,US,3,none
1,33,long_email,personalized,6,Monday,US,0,none
2,46,short_email,generic,14,Tuesday,US,3,none
3,49,long_email,personalized,11,Thursday,US,10,opened
4,65,short_email,generic,8,Wednesday,UK,3,none
...,...,...,...,...,...,...,...,...
99995,999969,short_email,generic,21,Thursday,US,0,none
99996,999972,long_email,personalized,6,Tuesday,US,5,none
99997,999976,long_email,personalized,5,Wednesday,UK,0,none
99998,999980,long_email,generic,10,Thursday,FR,4,none


In [32]:
df.to_csv("cleaned_data.csv")